#Data Preprocessing

In [1]:
!git clone https://github.com/indichealth/indic-health-demo.git

Cloning into 'indic-health-demo'...
remote: Enumerating objects: 231, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 231 (delta 55), reused 103 (delta 43), pack-reused 110
Receiving objects: 100% (231/231), 1.24 MiB | 7.26 MiB/s, done.
Resolving deltas: 100% (99/99), done.


In [9]:
%cd indic-health-demo/Dataset

/content
/content/indic-health-demo/Dataset


In [12]:
import pandas as pd
import os

In [ ]:
data_path = 'IHQID-WebMD'

In [ ]:
train = pd.read_csv(os.path.join(data_path, 'train.csv'))
test = pd.read_csv(os.path.join(data_path, 'test.csv'))

In [ ]:
train_queries = train['question_english']
test_queries = test['question_english']
train_intent = train['Manual_Intent']
test_intent = test['Manual_Intent']

# Model Creation

In [20]:
!python -m spacy download en_core_web_lg

2023-11-04 15:19:32.150096: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-04 15:19:32.150153: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-04 15:19:32.150190: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-04 15:19:34.120984: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [21]:
import spacy
import numpy as np

# Load the spacy model: nlp
nlp = spacy.load('en_core_web_lg')

In [22]:
# Calculate the dimensionality of nlp
embedding_dim = nlp.vocab.vectors_length
print(embedding_dim)

300


In [40]:
def encode_queries(queries):

    X = np.zeros((len(queries), embedding_dim))

    for idx, sentence in enumerate(queries):
        doc = nlp(sentence)
        X[idx, :] = doc.vector
    return X

train_emb = encode_queries(train_queries)
test_emb = encode_queries(test_queries)

In [41]:
from sklearn.preprocessing import LabelEncoder

def label_intents(intents):
    lenc = LabelEncoder()
    labels = lenc.fit_transform(intents)
    return labels

train_labels = label_intents(train_intent)
test_labels = label_intents(test_intent)

# Model Training

In [42]:
from sklearn.svm import SVC

def train_svc(embeddings, labels):
    clf = SVC(C = 1)
    clf.fit(embeddings, labels)
    return clf

model = train_svc(train_emb, train_labels)

# Model Evaluation

In [43]:
from sklearn.metrics import classification_report, confusion_matrix

def test_svc(model, embeddings, labels):

    pred_labels = model.predict(embeddings)

    # Count the number of correct predictions
    correct = 0
    for i in range(len(labels)):
      if pred_labels[i] == labels[i]:
        correct += 1

    print("Predicted {0} correctly out of {1} training examples".format(correct, len(labels)))
    return pred_labels

pred_labels = test_svc(model, test_emb, test_labels)
print(classification_report(test_labels, pred_labels))
print(confusion_matrix(test_labels, pred_labels))

Predicted 154 correctly out of 241 training examples
              precision    recall  f1-score   support

           0       0.69      0.66      0.68        76
           1       0.66      0.66      0.66        53
           2       0.58      0.77      0.66        73
           3       0.68      0.33      0.45        39

    accuracy                           0.64       241
   macro avg       0.65      0.60      0.61       241
weighted avg       0.65      0.64      0.63       241

[[50  5 18  3]
 [ 6 35 10  2]
 [ 8  8 56  1]
 [ 8  5 13 13]]
